# Exploratory Data Analysis in SQL
Use this workspace to take notes, store sample queries, and build your own interactive cheat sheet! 

You will need to connect your [SQL cells](https://workspace-docs.datacamp.com/work/sql-cell) to an integration to run a query.
- You can use a sample integration from the dropdown menu. This includes the **Course Databases** integration, which contains tables you used in our SQL courses.
- You can connect your own integration by following the instructions provided [here](https://workspace-docs.datacamp.com/integrations/what-is-an-integration).

## Common issues
- Error codes
    - Examples: 9, 99,-99
- Missing value codes: 
    - NA, NaN, N/A, #N/A
    - 0 = missing or 0?
- Outlier (extreme) values
    - Really high or low?
    - Negative values?
- Not really a number
    - Examples: zip codes, survey response categorie

# CHAPTER 1: WHAT IS DATABASE?

**Start exploring a database by **
- Identifying the tables and the **`foreign keys`** that link them.
- Look for **`missing values`**
- Count **`the number of observations`**
- Join tables to understand how they're related. 
- Learn about coalescing and casting data along the way.

**BASIC COMMAN FOR EDA** 
- **Couts no. of rows**: `count(*)`
- **Count no. of columns**: `SELECT count(*) FROM information_schema.columns WHERE table_name = '---';`
- **Count missing values**: `count(*) - count(column)`

## 1.1 Explore Table Sizes 

Let's start by exploring five related tables:
- **stackoverflow**: questions asked on Stack Overflow with certain tags
- **company**: information on companies related to tags in stackoverflow
- **tag_company**: links stackoverflow to company
- **tag_type**: type categories applied to tags in stackoverflow
- **fortune500**: information on top US companies

In [ ]:
-- 1. COUNT NUMBER OF ROWS
SELECT count(*) FROM company;

-- 2. COUNT NUMBER OF COLUMNS
SELECT count(*) FROM information_schema.columns WHERE table_name = 'company';

-- 3. COUNT MISSING VALUES

-- Select the count of ticker, 
-- subtract from the total number of rows, 
-- and alias as missing
SELECT count(*) - count(ticker) AS missing
  FROM fortune500;
  
-- Select the count of profits_change, 
-- subtract from total number of rows, and alias as missing
SELECT count(*) - count(profits_change) as missing
FROM fortune500;

-- Select the count of industry, 
-- subtract from total number of rows, and alias as missing
SELECT count(*) - count(industry) as missing
FROM fortune500;

Error: WITH datacamp_workspace__user_query AS (
  -- 1. COUNT NUMBER OF ROWS
  SELECT count(*) FROM company;
  
  -- 2. COUNT NUMBER OF COLUMNS
  SELECT count(*) FROM information_schema.columns WHERE table_name = 'company';
  
  -- 3. COUNT MISSING VALUES
  
  -- Select the count of ticker, 
  -- subtract from the total number of rows, 
  -- and alias as missing
  SELECT count(*) - count(ticker) AS missing
    FROM fortune500;
    
  -- Select the count of profits_change, 
  -- subtract from total number of rows, and alias as missing
  SELECT count(*) - count(profits_change) as missing
  FROM fortune500;
  
  -- Select the count of industry, 
  -- subtract from total number of rows, and alias as missing
  SELECT count(*) - count(industry) as missing
  FROM fortune500
)

SELECT * FROM datacamp_workspace__user_query LIMIT 100 - syntax error at or near ";"

### 1.2 Join Tables 
- Part of exploring a database is figuring out how tables relate to each other. The company and fortune500 tables don't have a formal relationship between them in the database, but this doesn't prevent you from joining them.

- To join the tables, you need to find a column that they have in common where the values are consistent across the tables.

### 1.3 Foreign keys 
Recall that foreign keys reference another row in the database via a unique ID. Values in a foreign key column are restricted to values in the referenced column OR **NULL**.

In [ ]:
-- Question 1: What is the most common 'stackoverflow' 'tag_type'? What companies have a 'tag' of that type?

-- Count the number of tags with each type
SELECT type, COUNT(tag) AS count
  FROM tag_type
 -- To get the count for each type, what do you need to do?
 GROUP BY type
 -- Order the results with the most common
 -- tag types listed first
 ORDER BY COUNT(tag) DESC;
 
 -- The most common tag type is 'Cloud' --
 
 -- Select the 3 columns desired
SELECT company.name, tag_type.tag, tag_type.type
  FROM company
  	   -- Join to the tag_company table
       INNER JOIN tag_company 
       ON company.id = tag_company.company_id
       -- Join to the tag_type table
       INNER JOIN tag_type
       ON tag_company.tag = tag_type.tag
  -- Filter to most common type
  WHERE type='cloud';

Error: WITH datacamp_workspace__user_query AS (
  -- Question 1: What is the most common 'stackoverflow' 'tag_type'? What companies have a 'tag' of that type?
)

SELECT * FROM datacamp_workspace__user_query LIMIT 100 - syntax error at or near ")"

### 1.4 'Coalesce' Function and Coalescde with self-join
The `coalesce()` function can be useful for specifying a default or backup value when a column contains NULL values.
`coalesce()` checks arguments in order and returns the first non-NULL value, if one exists.

- `coalesce(NULL, 1, 2) = 1`
- `coalesce(NULL, NULL) = NULL`
- `coalesce(2, 3, NULL) = 2`
In the `fortune500` data, `industry` contains some missing values. Use `coalesce()` to use the value of `sector` as the industry when `industry` is `NULL`. Then find the most common industry.

In [ ]:
-- In the fortune500 data, industry contains some missing values. Use coalesce() to use the value of sector as the industry when industry is NULL. Then find the most common industry.

-- Use coalesce
SELECT coalesce(industry, sector, 'Unknown') AS industry2,
       -- Don't forget to count!
       COUNT(*)
  FROM fortune500 
-- Group by what? (What are you counting by?)
 GROUP BY industry2
-- Order results to see most common first
 ORDER BY COUNT(*) DESC
-- Limit results to get just the one value you want
 LIMIT 1;

### 1.5 Effects of casting

When you cast data from one type to another, information can be lost or changed. See how the casting changes values and practice casting data using the CAST() function and the :: syntax.

`SELECT CAST(value AS new_type);` == `SELECT value::new_type;`

# CHAPTER 2: NUMERIC DATA TYPES & SUMMARY FUCTIONS

More exploration with: 
- **Summary functions**: `max`, `min`, `avg`, `stddev_samp()`, `stddev_pop()`,`var_pop()`, `var_samp()`
- **Explore with division**: for example, `unanswered_count/question_count::numeric`
- **Explor with distribution:** 
    - `trunc()` truncates numbers by replacing lower place value digits with zeros
    - `generate_series(from, to, step)` 
- **More summary functions:**
    - `corr(col1,col2 )`: correlation between two columns
    - `percentile_disc(percentile) WITHIN GROUP (ORDER BY column_name)`: median at 50 percentile

### 1.1 EXPLORE WITH DIVISION & DISTRIBUTION
- **Explore with division**: for example, `unanswered_count/question_count::numeric`
- **Explor with distribution:** 
    - `trunc()` truncates numbers by replacing lower place value digits with zeros
    - `generate_series(from, to, step)` 

In [ ]:
-- QUESTION: What information does the 'unanswered_pct' column in the stackoverflow table contain?

-- Divide unanswered_count by question_count
SELECT unanswered_count/question_count::numeric AS computed_pct, 
       -- What are you comparing the above quantity to?
       unanswered_pct
  FROM stackoverflow
 -- Select rows where question_count is not 0
 WHERE question_count != 0 
 LIMIT 10;
 
 -- ANSWER:  "unanswered_pct" is the percent of unanswered questions on Stack Overflow with the tag, not the percent of questions with the tag that are unanswered.

Error: -- What information does the 'unanswered_pct' column in the stackoverflow table contain?
-- Divide unanswered_count by question_count
SELECT unanswered_count/question_count::numeric AS computed_pct, 
       -- What are you comparing the above quantity to?
       unanswered_pct
  FROM stackoverflow
 -- Select rows where question_count is not 0
 WHERE question_count != 0 
 LIMIT 10;
 
 -- ANSWER:  "unanswered_pct" is the percent of unanswered questions on Stack Overflow with the tag, not the percent of questions with the tag that are unanswered. - relation "stackoverflow" does not exist

In [ ]:
-- QUESTION: Summarize the distribution of the number of questions with the tag "dropbox" on Stack Overflow per day by binning the data.

-- Bins created in Step 2
WITH bins AS (
      SELECT generate_series(2200, 3050, 50) AS lower,
             generate_series(2250, 3100, 50) AS upper),
     -- Subset stackoverflow to just tag dropbox (Step 1)
     dropbox AS (
      SELECT question_count 
        FROM stackoverflow
       WHERE tag='dropbox') 
-- Select columns for result
-- What column are you counting to summarize?
SELECT lower, upper, count(question_count) 
  FROM bins  -- Created above
       -- Join to dropbox (created above), 
       -- keeping all rows from the bins table in the join
       LEFT JOIN dropbox
       -- Compare question_count to lower and upper
         ON question_count >= lower 
        AND question_count < upper
 -- Group by lower and upper to count values in each bin
 GROUP BY lower, upper
 -- Order by lower to put bins in order
 ORDER BY lower;
 
 -- Outcome: 18 rows affected 

### 1.2 MORE SUMMARY FUNCTIONS
- `corr(col1,col2 )` correlation between two columns
- `percentile_disc(percentile) WITHIN GROUP (ORDER BY column_name)`: median at 50 percentile

In [ ]:
-- Compute the mean (avg()) and median assets of Fortune 500 companies by sector.

-- What groups are you computing statistics by?
SELECT sector,
       -- Select the mean of assets with the avg function
       avg(assets) AS mean,
       -- Select the median
       percentile_disc(0.5) WITHIN GROUP (ORDER BY assets) AS median
  FROM fortune500
 -- Computing statistics for each what?
 GROUP BY sector
 -- Order results by a value of interest
 ORDER BY mean;

### 1.3 TEMP TABLE 
- **Create TEMP TABLE Syntax** 
##### -- Create table as 
`CREATE TEMP TABLE new_tablename AS`
##### -- Query results to store in the table
`SELECT column1, column2`
`FROM table;`

- **Select Into Syntax** 
#### -- Select existing columns
`SELECT column1, column2`
##### -- Clause to direct results to a new temp table
`INTO TEMP TABLE new_tablename`
-- Existing table with exisitng columns
`FROM table;`

In [ ]:
-- QUESTION: How many questions had each tag on the first date for which data for the tag is available?
-- QUESTION: How many questions had the tag on the last day?
-- QUESTION: Also, compute the difference between these two values.
-- To clear table if it already exists
DROP TABLE IF EXISTS startdates;

CREATE TEMP TABLE startdates AS
SELECT tag, min(date) AS mindate
  FROM stackoverflow
 GROUP BY tag;
 
-- Select tag (Remember the table name!) and mindate
SELECT startdates.mindate, 
       -- Select question count on the min and max days
	   so_min.question_count AS min_date_question_count,
       so_max.question_count AS max_date_question_count,
       -- Compute the change in question_count (max- min)
       so_max.question_count - so_min.question_count AS change
  FROM startdates
       -- Join startdates to stackoverflow with alias so_min
       INNER JOIN stackoverflow AS so_min
          -- What needs to match between tables?
          ON startdates.tag = so_min.tag
         AND startdates.mindate = so_min.date
       -- Join to stackoverflow again with alias so_max
       INNER JOIN stackoverflow AS so_max
       	  -- Again, what needs to match between tables?
          ON startdates.tag = so_max.tag
         AND so_max.date = '2018-09-25';


### 1.4 INSERT INTO TEMP TABLE
- `INSERT INTO` top_companies
- `SELECT` rank, title
- `FROM` fortune500
- `WHERE` rank `BETWEEN` 11 AND 20;

# CHAPTER 3: Character data types and common issues

- **DATABASE: evanston311**
    - `street`column:  Street name
    - `description` column:  the details of the inquiry
    - `category` column:  the column groups inquiries into different types.

## 3.1  Character data types and common issues

- **Database: evanston311**
- Using `count(*)` and `GROUP BY` to count categories
- Using `LIKE` to find out extra space in the beginning or end of values 

## 3.2 ISSUE 2: Cases and Spaces

1. **Case sensitivity`'apple' != 'Apple'`**
    - `lower()` OR `upper()` for converting case
3. **Spaces count: `' apple' != 'apple'` OR `'' != ' '`**
    - Using `LIKE '%apple%'` OR `IKIKE` for case insensitive
    - `trim(column_name, char to be trimmed)`
5. **Empty strings aren't null: `'' != NULL`**
6. **Punctuation differences: `'to-do' != 'to–do'`**


In [ ]:
-- Any issues in the 'Street' name? 
-- Remove the house numbers, extra punctuation, and any spaces from the beginning and end of the street values as a first attempt at cleaning up the values.
SELECT distinct street,
       -- Trim off unwanted characters from street
       trim(street, '0123456789 #/.') AS cleaned_street
  FROM evanston311
 ORDER BY street;

In [ ]:
-- Find the most common categories for rows with a description about trash that don't have a trash-related category.
-- Count rows with each category
SELECT category, count(*)
  FROM evanston311 
 WHERE (description ILIKE '%trash%'
    OR description ILIKE '%garbage%') 
   AND category NOT LIKE '%Trash%'
   AND category NOT LIKE '%Garbage%'
 -- What are you counting?
 GROUP BY category
 --- order by most frequent values
 ORDER BY count(*) DESC
 LIMIT 10;

## 3.3 ISSUE 3 - Bring order to messy text

### Splitting and concatenating text
- **Choose the first n characters: `left(column_name, n)`**
- **Choose the last n characters: `right(column_name, n)`**
- **Choose substring from specific position: `substring(string FROM start FOR length);`**
- **Splitting on a delimiter: `SELECT split_part(string, delimiter, part)`;**
- **Concatenating text: using `concat()` OR `||` OR `concat_ws(separator, string 1, string 2, ...)` ** 
    - `SELECT concat('a', NULL,'cc');` -- result: **acc**
    - `SELECT concat('a', 2,'cc');` -- result: a2cc
    - `SELECT 'a' || 2 || 'cc'`;  - result: a2cc
    - `SELECT 'a' || NULL || 'cc';` -- result: (empty)


In [ ]:
-- QUESTION: Extract just the first word of each street value to find the most common streets regardless of the suffix.
-- Select the first word of the street value
SELECT split_part(street,' ', 1) AS street_name, 
       count(*)
  FROM evanston311
 GROUP BY street_name
 ORDER BY count DESC
 LIMIT 20;

In [ ]:
-- QUESTION: Select the first 50 characters of description when description starts with the word "I".

-- Select the first 50 chars when length is greater than 50
SELECT CASE WHEN length(description) > 50
            THEN left(description, 50) || '...'
       -- otherwise just select description
       ELSE description
       END
  FROM evanston311
 -- limit to descriptions that start with the word I
 WHERE description LIKE 'I %'
 ORDER BY description;

### 3.4 ISSUE 4: MULTIPLE TRANSFORMATIONS
- **Step 1: `CREATE TEMP TABLE`**
- **Step 2: `UPDATE` values **
    - `UPDATE`** table_name**
        `SET` **column_name = new_value**
         `WHERE` **condition** 
- **Step 3: `JOIN` original and `TEMP` table**

In [ ]:
-- QUESTION: Choose the main category 'Trach Cart', 'Snow removal' and count of each 

-- Step 1: CREATE TEMP TABLE
DROP TABLE IF EXISTS recode;
CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;
-- Step 2: UPDATE THE TEMP TABLE
UPDATE recode SET standardized='Trash Cart' 
 WHERE standardized LIKE 'Trash%Cart';
UPDATE recode SET standardized='Snow Removal' 
 WHERE standardized LIKE 'Snow%Removal%';
UPDATE recode SET standardized='UNUSED' 
 WHERE standardized IN ('THIS REQUEST IS INACTIVE...Trash Cart', 
               '(DO NOT USE) Water Bill',
               'DO NOT USE Trash', 'NO LONGER IN USE');

-- Step 3: JOIN BOTH TABLES 
-- Select the recoded categories and the count of each
SELECT standardized, count(*)
-- From the original table and table with recoded values
  FROM evanston311 
       INNER JOIN recode 
       -- What column do they have in common?
       ON evanston311.category = recode.category 
 -- What do you need to group by to count?
 GROUP BY standardized
 -- Display the most common val values first
 ORDER BY count(*) DESC;

Tips and Tricks: **Create TEMP TABLE wirh indivator variables**
- **Using `cast(col_name LIKE string as integer)` - `LIKE` produces `True` or `False` as a result, but casting a boolean (True or False) as an `integer`**

In [ ]:
-- To clear table if it already exists
DROP TABLE IF EXISTS indicators;

-- Create the indicators temp table
CREATE TEMP TABLE indicators AS
  -- Select id
  SELECT id, 
         -- Create the email indicator (find @) 
         CAST (description LIKE '%@%' AS integer) AS email,
         -- Create the phone indicator
         CAST(description LIKE '%___-___-____%' AS integer) AS phone 
    -- What table contains the data? 
    FROM evanston311;

-- Select the column you'll group by
SELECT priority,
       -- Compute the proportion of rows with each indicator
       sum(email)/count(*)::numeric AS email_prop, 
       sum(phone)/count(*)::numeric AS phone_prop
  -- Tables to select from
  FROM evanston311
       LEFT JOIN indicators
       -- Joining condition
       ON evanston311.id=indicators.id
 -- What are you grouping by?
 GROUP BY priority;

# CHAPTER 4: DATE/TIME TYPE AND FORMATS

## 4.1 Main Date & Time Types 
- **date**: `YYYY-MM-DD`. Example: `2018-12-30`
- **timestamp**: `YYYY-MM-DD HH:MM:SS`, for example: `2018-12-30 13.03`
- **interval**: `6 days 01:48:08`, `00:51:03`,`1 day 21:57:47`
- **Timestamp with timezone**: `YYYY-MM-DD HH:MM:SS+HH`. Example: `2004-10-19 10:23:54+02`

## 4.2 Date and time comparisons 
- Compare with `>` , `<` , `=`. Example: `SELECT '2018-01-01' > '2017-12-31';`
- `now()` : current timestamp. Example: `SELECT now() > '2017-12-31';`

#### 4.2.1 Date Addition
- `SELECT '2018-12-10'::date + '1 year 2 days 3 minutes'::interval ;`
    - Result: `2019-12-12 00:03:00`
#### 4.2.2 Date Subtraction 
- `SELECT now() - '2015-01-01';`
    - Result: `1439 days 21:32:22.616076`

In [ ]:
-- Add 100 days to the current timestamp
SELECT now(), now() +'100 days'::interval;

-- Select the current timestamp, 
-- and the current timestamp + 5 minutes
SELECT now(), now() + '5 minutes'::interval;

-- Select the category and the average completion time by category
SELECT category, 
       AVG(date_completed - date_created) AS completion_time
  FROM evanston311
 GROUP BY category
-- Order the results
 ORDER BY completion_time DESC;

## 4.3 Date/time components and aggregation

### 4.3.1 Common date/time fields

- `century`: 2019-01-01 = `century 21 `
- decade: 2019-01-01 = `decade 201` 
- `year`, `month`, `day` 
- `hour`, `minute`, `second`
- `week` 
- `dow`: day of week

### 4.3.2 Date/time Components and Aggregation

- **Extracting Function:** `date_part('field', timestamp)` OR `EXTRACT(FIELD FROM timestamp)`
- **Getting the name of the day of the week**: `to_char(date_created, 'day')`
- **Truncating dates:** `date_trunc('field', timestamp)`
    - Example: **SELECT date_trunc('month', now());**. Result: `2018-12-01 00:00:00`

In [ ]:
-- Variation by day of week

-- Select name of the day of the week the request was created 
SELECT to_char(date_created, 'day') AS day, 
       -- Select avg time between request creation and completion
       AVG(date_completed - date_created) AS duration
  FROM evanston311 
 -- Group by the name of the day of the week and 
 -- integer value of day of week the request was created
 GROUP BY day, EXTRACT(DOW FROM date_created)
 -- Order by integer value of the day of the week 
 -- the request was created
 ORDER BY EXTRACT(DOW FROM date_created);

Error: // Variation by day of week

-- Select name of the day of the week the request was created 
SELECT to_char(date_created, 'day') AS day, 
       -- Select avg time between request creation and completion
       AVG(date_completed - date_created) AS duration
  FROM evanston311 
 -- Group by the name of the day of the week and 
 -- integer value of day of week the request was created
 GROUP BY day, EXTRACT(DOW FROM date_created)
 -- Order by integer value of the day of the week 
 -- the request was created
 ORDER BY EXTRACT(DOW FROM date_created); - syntax error at or near "//"

In [ ]:
-- Aggregate daily counts by month
SELECT date_trunc('month', day) AS month,
       avg(count)
  -- Subquery to compute daily counts
  FROM (SELECT date_trunc('day',date_created) AS day,
               count(*) AS count
          FROM evanston311
         GROUP BY day) AS daily_count
 GROUP BY month
 ORDER BY month;

## 4.4 Aggregating withdate/time series

### 4.4.1 Generate series
- **Syntax:** `SELECT generate_series(from, to, interval);`
- **Generate series from he beginning**
    - `SELECT generate_series('2018-01-31','2018-12-31','1 month'::interval);` 
    - -- Subtract 1 day to get end of month
SELECT generate_series('2018-02-01',
-- start 1 month late
'2019-01-01'
,
'1 month'::interval) -
'1 day'::interval